In [ ]:
!pip install selenium
!pip install BeautifulSoup4
!pip install requests
!pip install webdriver-manager
!pip install transformers
!pip install torch
!pip install streamlit --quiet

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# caminho do modelo BERT com fine tuning salvo anteriormente
caminho_no_drive = '/content/drive/MyDrive/MODELOS/modelo_multirotulomulticlasse_vf10.pth'

# copie o arquivo para o ambiente de execução do Colab
caminho_no_colab = '/content/modelo.pth'
!cp "$caminho_no_drive" "$caminho_no_colab"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile app.py

import streamlit as st
from PIL import Image
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import requests
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

st.set_page_config(layout="centered", page_icon=":earth_americas:",initial_sidebar_state="collapsed")

st.set_option('deprecation.showPyplotGlobalUse', False)

def crawler(input_pesquisa, avaliacoes):

    servico = Service(ChromeDriverManager().install())

    navegador = webdriver.Chrome(service=servico)

    navegador.get("https://www.tripadvisor.com/Restaurants")

    search_box = navegador.find_element(By.XPATH,"/html/body/div[2]/div/div[1]/div/div/div/div/div/div[2]/div/div/div[2]/div/form/div/div/input")

    search_box.send_keys(input_pesquisa)

    search_box.send_keys(Keys.RETURN)

    navegador.implicitly_wait(50)

    elemento = navegador.find_element(By.CLASS_NAME, "result-title")

    navegador.execute_script("arguments[0].click();", elemento)

    navegador.implicitly_wait(50)

    janelas = navegador.window_handles

    navegador.switch_to.window(janelas[0])

    navegador.implicitly_wait(50)

    navegador.close()

    navegador.switch_to.window(janelas[1])

    navegador.implicitly_wait(50)

    elemento_span = navegador.find_element(By.CLASS_NAME, "taLnk.ulBlueLinks")

    navegador.execute_script("arguments[0].click();", elemento_span)

    navegador.implicitly_wait(50)

    label_element = navegador.find_element(By.CSS_SELECTOR,'label[for="filters_detail_language_filterLang_pt"]')

    label_element.click()

    navegador.implicitly_wait(50)

    select_element = navegador.find_element(By.ID, "sort-by")

    select = Select(select_element)

    select.select_by_value("ml_sorted")

    navegador.implicitly_wait(50)

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}

    lang_atual = navegador.execute_script("return document.documentElement.lang;")

    navegador.execute_script('document.documentElement.lang = "pt-BR";')

    navegador.implicitly_wait(50)

    lang_modificado = navegador.execute_script("return document.documentElement.lang;")

    html_pagina = navegador.page_source

    soup = BeautifulSoup(html_pagina, 'html.parser')

    reviews = soup.find_all('div', class_='entry')

    for review in reviews:

        review_text1 = review.find('p', class_='partial_entry').text

        avaliacoes.append(review_text1)

        #print(avaliacoes) - lista com todas avaliações

    navegador.implicitly_wait(50)

    navegador.quit()

    return avaliacoes

def load_modelo_e_analisa(avaliacoes):

  pth_file_path = '/content/modelo.pth'

  checkpoint = torch.load(pth_file_path, map_location=torch.device('cpu'))

  model_state_dict = checkpoint['model_state_dict']

  tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)
  model = BertForSequenceClassification.from_pretrained('neuralmind/bert-base-portuguese-cased', num_labels=3)
  model.classifier = nn.Linear(model.config.hidden_size, 6)
  model.load_state_dict(model_state_dict)

  model.eval()

  colunas = ['Comentário', 'Ambiente','Comida','Geral','Preço','Serviço','Segurança']
  dataframe_resultados = pd.DataFrame(columns=colunas)

  for text in avaliacoes:

    df1 = pd.DataFrame([text])

    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    with torch.no_grad():
      outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=1).tolist()[0]

    class_mapping = {0: 'ambiente', 1: 'comida', 2: 'geral', 3: 'preco', 4: 'servico', 5: 'seguranca'}
    sentiment_labels = {class_name: int(prob * 3) for class_name, prob in zip(class_mapping.values(), probabilities)}

    df2 = pd.DataFrame([sentiment_labels])

    df3 = pd.concat([df1,df2], axis = 1)
    df3.columns = ['Comentário', 'Ambiente','Comida','Geral','Preço','Serviço','Segurança']

    dataframe_resultados = dataframe_resultados.append(df3, ignore_index=True)

  return dataframe_resultados


class SessionState:
    def __init__(self):
        self.custom_primary_color = "#0E8388"
        self.custom_background_color = "#2C3333"
        self.custom_secondary_background_color = "#2E4F4F"
        self.custom_text_color = "#CBE4DE"
        self.custom_font = "monospace"

ss = SessionState()

custom_style = f"""
    <style>
    .stApp {{
        background-color: {ss.custom_background_color};
        color: {ss.custom_text_color};
        font-family: {ss.custom_font};
    }}
    .stButton {{
        background-color: {ss.custom_primary_color};
    }}
    .css-3ewlls {{
        background-color: {ss.custom_secondary_background_color};
    }}
    </style>
"""

st.markdown(custom_style, unsafe_allow_html=True)

st.write("""
    <style>
    .pageTitle {
        color: #f0f0f0;
        text-align: center;
    }
    </style>
    <h1 class="pageTitle">GastronoTripReview</h1>
""", unsafe_allow_html=True)


st.header("",divider='green')


st.markdown(
    """
    <p style="color: white; font-size: 20px;text-align: center;">Consulte como está a reputação do seu restaurante através de um resumo fácil e prático decorrente da análise de sentimentos de reviews feitas por <strong style="font-weight: bold;">turistas e clientes reais</strong> na plataforma TripAdvisor. <span class="emoji">&#x1F5FA;</span> <span class="emoji">&#x1F374;</span></p>
    """,
    unsafe_allow_html=True,
)

image_url1 = "https://raw.githubusercontent.com/isalbineli/testes/main/smileys.jpg"



st.markdown(
    """
    <style>
    .sidebar .sidebar-content {
        background-color: #5c4f5c;
    }
    </style>
    """,
    unsafe_allow_html=True,
)

image_url2 = "https://raw.githubusercontent.com/isalbineli/testes/main/shutterstock_1756325360.jpg"

st.sidebar.image(image_url2, caption="Suas reviews como você nunca viu :)")

with st.form("chat_form"):
    st.markdown(
    """
    <p style="color: white;">Insira o nome do restaurante para consulta:</p>
    """,
    unsafe_allow_html=True,)
    input_pesquisa = st.text_input("")
    submit_button = st.form_submit_button("Analisar",type="primary",use_container_width=True)

if submit_button:

    avaliacoes = []
    avaliacoes = crawler(input_pesquisa, avaliacoes)

    #st.write(avaliacoes)
    st.success("Acabamos de capturar todas as reviews...")
    st.info("Realizando as análises...\n")
    df_resultante = load_modelo_e_analisa(avaliacoes)
    st.success("Análises realizadas.")

    st.markdown(
    """
    <p style="color: white; font-size: 20px;text-align: center;">Reviews e Análise de Sentimentos por Classe</p>
    """,
    unsafe_allow_html=True,
    )

    st.dataframe(df_resultante, use_container_width=True)

    st.markdown(
    """
    <p style="color: orange; font-size: 15px;text-align: center;">Legenda</p>
    """,
    unsafe_allow_html=True,
    )

    data = {
    'Negativo': [0],
    'Positivo': [1],
    'Neutro': [2],
    }

    legenda = pd.DataFrame(data)

    st.dataframe(legenda,use_container_width=True)


    st.markdown(
    """
    <p style="color: white; font-size: 20px;text-align: center;">Gráficos com as Contagens de Sentimentos Identificados por Classe</p>
    """,
    unsafe_allow_html=True,
    )

    if 'Comentário' in df_resultante.columns:
        df_resultante = df_resultante.drop(columns=['Comentário'])

    num_cols = len(df_resultante.columns)
    fig, axs = plt.subplots(1, num_cols, figsize=(15, 4))


    for i, col in enumerate(df_resultante.columns):
        ax = axs[i]
        sns.countplot(x=df_resultante[col], ax=ax, palette='viridis')
        ax.set_xticks([0, 1, 2])
        ax.set_xticklabels(['negativo', 'positivo', 'neutro'])
        ax.set_title(f'{col}')

    plt.tight_layout()

    st.pyplot()

    st.markdown(
    """
    <p style="color: white; font-size: 20px;text-align: center;">Resultado Geral por Classe</p>
    """,
    unsafe_allow_html=True,
    )

    a = (pd.DataFrame([df_resultante['Ambiente'].mode()])).transpose()
    b = (pd.DataFrame([df_resultante['Comida'].mode()])).transpose()
    c = (pd.DataFrame([df_resultante['Geral'].mode()])).transpose()
    d = (pd.DataFrame([df_resultante['Preço'].mode()])).transpose()
    e = (pd.DataFrame([df_resultante['Serviço'].mode()])).transpose()
    f = (pd.DataFrame([df_resultante['Segurança'].mode()])).transpose()

    df_moda = pd.concat([a,b,c,d,e,f], axis = 1)

    st.dataframe(df_moda, use_container_width=True)

    insere_figuras1 = df_moda["Ambiente"].tolist()
    insere_figuras2 = df_moda["Comida"].tolist()
    insere_figuras3 = df_moda["Geral"].tolist()
    insere_figuras4 = df_moda["Preço"].tolist()
    insere_figuras5 = df_moda["Serviço"].tolist()
    insere_figuras6 = df_moda["Segurança"].tolist()

    positivo = "https://raw.githubusercontent.com/isalbineli/testes/main/positivo.png"
    negativo = "https://raw.githubusercontent.com/isalbineli/testes/main/negativo.png"
    neutro = "https://raw.githubusercontent.com/isalbineli/testes/main/neutro.png"

    col1, col2, col3, col4, col5, col6 = st.columns(6)

    if insere_figuras1[0] == 0:
      col1.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Ambiente</p>    """,    unsafe_allow_html=True,    )
      col1.image(negativo, caption="Ambiente",width=100)

    if insere_figuras1[0] == 1:
      col1.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Ambiente</p>    """,    unsafe_allow_html=True,    )
      col1.image(positivo, caption="Ambiente",width=100)

    if insere_figuras1[0] == 2:
      col1.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Ambiente</p>    """,    unsafe_allow_html=True,    )
      col1.image(neutro, caption="Ambiente",width=100)

    if insere_figuras2[0] == 0:
      col2.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Comida</p>    """,    unsafe_allow_html=True,     )
      col2.image(negativo, caption="Comida",width=100)

    if insere_figuras2[0] == 1:
      col2.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Comida</p>    """,    unsafe_allow_html=True,     )
      col2.image(positivo, caption="Comida",width=100)

    if insere_figuras2[0] == 2:
      col2.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Comida</p>    """,    unsafe_allow_html=True,     )
      col2.image(neutro, caption="Comida",width=100)

    if insere_figuras3[0] == 0:
      col3.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Geral</p>    """,    unsafe_allow_html=True,     )
      col3.image(negativo, caption="Geral",width=100)

    if insere_figuras3[0] == 1:
      col3.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Geral</p>    """,    unsafe_allow_html=True,     )
      col3.image(positivo, caption="Geral",width=100)

    if insere_figuras3[0] == 2:
      col3.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Geral</p>    """,    unsafe_allow_html=True,     )
      col3.image(neutro, caption="Geral",width=100)

    if insere_figuras4[0] == 0:
      col4.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Preço</p>    """,    unsafe_allow_html=True,     )
      col4.image(negativo, caption="Preço",width=100)

    if insere_figuras4[0] == 1:
      col4.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Preço</p>    """,    unsafe_allow_html=True,     )
      col4.image(positivo, caption="Preço",width=100)

    if insere_figuras4[0] == 2:
      col4.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Preço</p>    """,    unsafe_allow_html=True,     )
      col4.image(neutro, caption="Preço",width=100)

    if insere_figuras5[0] == 0:
      col5.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Serviço</p>    """,    unsafe_allow_html=True,     )
      col5.image(negativo, caption="Serviço",width=100)

    if insere_figuras5[0] == 1:
      col5.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Serviço</p>    """,    unsafe_allow_html=True,     )
      col5.image(positivo, caption="Serviço",width=100)

    if insere_figuras5[0] == 2:
      col5.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Serviço</p>    """,    unsafe_allow_html=True,     )
      col5.image(neutro, caption="Serviço",width=100)

    if insere_figuras6[0] == 0:
      col6.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Segurança</p>    """,    unsafe_allow_html=True,     )
      col6.image(negativo, caption="Segurança",width=100)

    if insere_figuras6[0] == 1:
      col6.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Segurança</p>    """,    unsafe_allow_html=True,     )
      col6.image(positivo, caption="Segurança",width=100)

    if insere_figuras6[0] == 2:
      col6.markdown(    """    <p style="color: white; font-size: 15px;text-align: center;">Segurança</p>    """,    unsafe_allow_html=True,     )
      col6.image(neutro, caption="Segurança",width=100)

st.image(image_url1, use_column_width=True)


Overwriting app.py


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.729s
your url is: https://beige-points-film.loca.lt
